In [1]:
import numpy as np
import scipy.integrate
scipy.integrate.trapz = np.trapz
import yaml
from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, TextInput, PreText, DataTable, Select
from bokeh.plotting import figure, curdoc
from bokeh.themes import Theme

from pytmatrix.tmatrix import Scatterer
from pytmatrix.psd import PSDIntegrator, GammaPSD
from pytmatrix import orientation, radar, tmatrix_aux, refractive

import pandas as pd
pd.options.display.float_format = '{:,.3f}'.format
from scipy.special import gamma


In [2]:
# calculations
def drop_ar(D_eq):
    if D_eq < 0.7:
        return 1.0;
    elif D_eq < 1.5:
        return 1.173 - 0.5165*D_eq + 0.4698*D_eq**2 - 0.1317*D_eq**3 - \
            8.5e-3*D_eq**4
    else:
        return 1.065 - 6.25e-2*D_eq - 3.99e-3*D_eq**2 + 7.66e-4*D_eq**3 - \
            4.095e-5*D_eq**4 


def get_scattering_props(Dm=2.0,logNw=3.0,mu=0.0,wavelength='10',cant=0.0,ptype='rain'):
    if ((wavelength == '5') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '3') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '10') & (ptype == 'hail')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=complex(1.78, 7.9e-4))
    elif ((wavelength == '5') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_C, m=refractive.m_w_10C[tmatrix_aux.wl_C])
    elif ((wavelength == '3') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_X, m=refractive.m_w_10C[tmatrix_aux.wl_X])
    elif ((wavelength == '10') & (ptype == 'rain')):
        scatterer = Scatterer(wavelength=tmatrix_aux.wl_S, m=refractive.m_w_10C[tmatrix_aux.wl_S])
    scatterer.psd_integrator = PSDIntegrator()
    if ptype == 'rain':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 1.0/drop_ar(D)
    if ptype == 'hail':
        scatterer.psd_integrator.axis_ratio_func = lambda D: 0.99
    scatterer.psd_integrator.D_max = 10.0
    scatterer.psd_integrator.geometries = (tmatrix_aux.geom_horiz_back, tmatrix_aux.geom_horiz_forw)
    if cant > 0.0:
        scatterer.or_pdf = orientation.gaussian_pdf(cant)
    scatterer.orient = orientation.orient_averaged_fixed
    scatterer.psd_integrator.init_scatter_table(scatterer)

    D0 = (3.67 + mu)/(4 + mu) * Dm
    Nw = 10.**logNw   
    scatterer.psd = GammaPSD(D0=D0, Nw=Nw, mu=mu)

    Zh = 10*np.log10(radar.refl(scatterer))
    Zv = 10*np.log10(radar.refl(scatterer, False))
    Zdr = 10.*np.log10(radar.Zdr(scatterer))
    Ldr = 10*np.log10(radar.ldr(scatterer))
    rho_hv = radar.rho_hv(scatterer)
    delta = radar.delta_hv(scatterer)
    scatterer.set_geometry(tmatrix_aux.geom_horiz_forw)
    Kdp = radar.Kdp(scatterer)
    Ah = radar.Ai(scatterer)
    Av = radar.Ai(scatterer,h_pol=False)

    f_u = (6/(4**4))*((4 + mu)**(mu + 4))/(gamma(mu+4))

    dsd_df = calc_dsd(Dm,logNw,mu)

    NT = Nw * f_u * gamma(mu + 1) * Dm / ((4 + mu)**(mu + 1))
    if ptype == 'rain':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 1000.)
    elif ptype == 'hail':
        LWC = (1. * np.pi * Nw * Dm**4) / (4.**4 * 917.)
        
#    Vt = -0.193 + 4.96*dsd_df['D'] - 0.904*dsd_df['D']**2 + 0.0566*dsd_df['D']**3
    
    Vt = 17.67 * dsd_df['D']**0.67
    
    R = np.sum(dsd_df['ND']*(np.pi*dsd_df['D']/6.)*Vt)/3600.



    integ_df = pd.DataFrame({'NT':[NT], 
                            'WC (g m-3)':[LWC], 
                            'Zh (dBZ)':[Zh], 
                            'Zv (dBZ)':[Zv], 
                            'Zdr (dB)': [Zdr], 
                            'Ldr (dB)': [Ldr], 
                            'rho_hv':[rho_hv], 
                            'Kdp (deg km-1)':[Kdp], 
                            'delta (deg km-1)':[delta],
                            'Ah (dBZ/km)':[Ah],
                            'Adr (dB/km)':[Ah-Av],
                            'R (mm/hr)': R})

    return dsd_df, integ_df



def calc_dsd(Dm,logNw,mu):
    f_u = (6/4**4)*((4 + mu)**(mu + 4))/(gamma(mu+4))
    dsd_df = pd.DataFrame()
    dsd_df['D'] = np.arange(0.1,20,0.1)
    dsd_df['ND'] = 10**logNw * f_u * ((dsd_df['D'] / Dm) ** mu) * np.exp(-1.*(4 + mu)*(dsd_df['D'] / Dm))
 
    return dsd_df


In [3]:
# Set up data
Dm = 1.5
logNw = np.log10(5800)
mu = 3
wavelength = '10'
cant = 0.0
ptype = 'rain'

dsd_df, integ_df = get_scattering_props(Dm=Dm, logNw=logNw, mu=mu, wavelength=wavelength, 
    cant=cant, ptype=ptype)

In [4]:
integ_df

,NT,WC (g m-3),Zh (dBZ),Zv (dBZ),Zdr (dB),Ldr (dB),rho_hv,Kdp (deg km-1),delta (deg km-1),Ah (dBZ/km),Adr (dB/km),R (mm/hr)
0,582.832,0.360,36.161,35.397,0.764,-35.282,0.999,0.086,0.001,0.003,0.000,13.156


In [5]:
!wget https://github.com/maahn/pyOptimalEstimation_examples/raw/master/data/huntsville_parameters.nc

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
import xarray as xr
data = xr.open_dataset('huntsville_parameters.nc', engine='netcdf4')

In [7]:
data.Nw.max

<bound method DataArrayAggregations.max of <xarray.DataArray 'Nw' (time: 18969)>
[18969 values with dtype=float64]
Coordinates:
  * time     (time) datetime64[ns] 2009-12-13T04:24:00 ... 2011-04-04T21:51:00
Attributes:
    units:    1/m3/mm>

In [8]:
def calc_dsd_all(Dm,Nw):
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    return integ_df

In [ ]:
from dask.distributed import Client
client = Client() 

2025-11-06 04:54:07,800 - tornado.application - ERROR - Uncaught exception GET /status/ws (127.0.0.1)
HTTPServerRequest(protocol='http', host='127.0.0.1:8787', method='GET', uri='/status/ws', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "c:\Users\tomy3\miniforge3\envs\logreg-weather-py39\lib\site-packages\tornado\web.py", line 1848, in _execute
    result = await result
  File "c:\Users\tomy3\miniforge3\envs\logreg-weather-py39\lib\site-packages\tornado\websocket.py", line 277, in get
    await self.ws_connection.accept_connection(self)
  File "c:\Users\tomy3\miniforge3\envs\logreg-weather-py39\lib\site-packages\tornado\websocket.py", line 890, in accept_connection
    await self._accept_connection(handler)
  File "c:\Users\tomy3\miniforge3\envs\logreg-weather-py39\lib\site-packages\tornado\websocket.py", line 973, in _accept_connection
    await self._receive_frame_loop()
  File "c:\Users\tomy3\miniforge3\envs\logreg-weather-py39\lib\site-packag

In [10]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 24,Total memory: 31.83 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51513,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 24
Started: Just now,Total memory: 31.83 GiB
Comm: tcp://127.0.0.1:59942,Total threads: 4
Dashboard: http://127.0.0.1:59943/status,Memory: 5.30 GiB
Nanny: tcp://127.0.0.1:51516,


In [11]:
import numpy as np
futures = client.map(calc_dsd_all,data.Dm.values,np.log10(data.Nw.values))

In [12]:
futures

[<Future: pending, key: calc_dsd_all-86ddb8d0c355ceac7a9f4011563ca410>,
 <Future: pending, key: calc_dsd_all-1a0871de6c259572a2e444b4a4f71d35>,
 <Future: pending, key: calc_dsd_all-ae096fb0be00fd1722dead11618677f2>,
 <Future: pending, key: calc_dsd_all-5a7790b445b82be3a93ee97d8c1a1ba2>,
 <Future: pending, key: calc_dsd_all-b3cdd55c0130aa6d96d12a53aac0d59f>,
 <Future: pending, key: calc_dsd_all-8f201240738e627b16a13be073ad87a0>,
 <Future: pending, key: calc_dsd_all-9122c3c6365346bea9f573f68b817098>,
 <Future: pending, key: calc_dsd_all-ea624434d977971896d2d5c32ecb8600>,
 <Future: pending, key: calc_dsd_all-b1c0d6437477ba3f57638cff3818f478>,
 <Future: pending, key: calc_dsd_all-fceaf22587d3ecd1f5728ba6af8641d0>,
 <Future: pending, key: calc_dsd_all-ff0b55666d9af4576383a8e4c1e227bf>,
 <Future: pending, key: calc_dsd_all-a7adf74fec896ad87afac71a9e8649f1>,
 <Future: pending, key: calc_dsd_all-2b4f88fac025aafaf94bc9b33139b871>,
 <Future: pending, key: calc_dsd_all-28f20c67be418b0bfbc69253b5f

In [13]:
#import os

#output_dir = os.path.expanduser("~/Downloads/out")
#os.makedirs(output_dir, exist_ok=True)

In [14]:
import pandas as pd

all_df = pd.DataFrame()
for i in range(data.time.size):
    print(i)
    dsd_df, integ_df = get_scattering_props(Dm=float(data.Dm[i]), logNw=float(np.log10(data.Nw[i])), 
                                            mu=np.random.normal(3,0.5), wavelength=wavelength, 
                                            cant=cant, ptype=ptype)
    integ_df.to_csv('~/Downloads/out/scatt_{:05d}.csv'.format(i))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [ ]:
float(data.Dm[i])